<a href="https://colab.research.google.com/github/UrviSoni/pytorch_deeplearning/blob/master/test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# google colab does not come with torch installed. And also, in course we are using torch 0.4. 
# so following snippet of code installs the relevant version

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [2]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [0]:
!wget -c https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip

In [0]:
!unzip flower_data.zip

In [3]:
#!pip install --no-cache-dir -I pillow

!pip uninstall -y Pillow
# install the new one
!pip install Pillow==5.3.0
# import the new one
import PIL
print(PIL.PILLOW_VERSION)
# this should print 5.3.0. If it doesn't, then restart your runtime:
# Menu > Runtime > Restart Runtime


Uninstalling Pillow-5.3.0:
  Successfully uninstalled Pillow-5.3.0
  Using cached https://files.pythonhosted.org/packages/62/94/5430ebaa83f91cc7a9f687ff5238e26164a779cca2ef9903232268b0a318/Pillow-5.3.0-cp36-cp36m-manylinux1_x86_64.whl
5.3.0


In [0]:
# Imports here
import matplotlib.pyplot as plt
import seaborn as sb

#%reload_ext autoreload        <------------— comment out 
#%autoreload 0                 <------------— comment out
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from torch.autograd import Variable
import json
from collections import OrderedDict

import math

plt.ion() 

In [0]:
data_dir = '/content/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
!git clone https://github.com/udacity/pytorch_challenge.git
import json

with open('pytorch_challenge/cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)


In [0]:
# Building and Training the Network

data_dir = 'flower_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/valid', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)


In [7]:
model = models.resnet152(pretrained=True)
model


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet152(pretrained=True)


# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
classifier = nn.Sequential(nn.Linear(2048 , 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 102),
                                 nn.LogSoftmax(dim=1))
model.fc= classifier
criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
model.to(device);


In [13]:
from tqdm import tqdm

epochs = 60
steps = 0

torch.device('cuda')
train_losses, test_losses = [], []
for e in range(epochs):
    loader = tqdm(trainloader, total=len(trainloader))
    running_loss = 0
    for batch_idx,(images, labels) in enumerate(loader):
        images,labels=images.cuda(),labels.cuda()
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        #print(loss)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        #
        running_loss += loss.item()
        #print("haha,Its me")
        
    
    
    ## TODO: Implement the validation pass and print out the validation accuracy
    model.eval()
    test_loss=0
    accuracy=0
    for images, labels in testloader:
      images,labels=images.cuda(),labels.cuda()
      log_ps = model(images)
      test_loss += criterion(log_ps, labels)
                
      ps = torch.exp(log_ps)
      top_p, top_class = ps.topk(1, dim=1)
      equals = top_class == labels.view(*top_class.shape)
      accuracy += torch.mean(equals.type(torch.FloatTensor))
         
      train_losses.append(running_loss/len(trainloader))
      test_losses.append(test_loss/len(testloader))

    print("Epoch: {}/{}.. ".format(e+1, epochs),
          "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
           "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
           "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))




  0%|          | 0/103 [00:00<?, ?it/s]

Epoch: 1/60..  Training Loss: 1.388..  Test Loss: 0.756..  Test Accuracy: 0.830


  0%|          | 0/103 [00:00<?, ?it/s]

Epoch: 2/60..  Training Loss: 0.858..  Test Loss: 0.537..  Test Accuracy: 0.875


  0%|          | 0/103 [00:00<?, ?it/s]

...

Epoch: 11/60..  Training Loss: 0.345..  Test Loss: 0.254..  Test Accuracy: 0.938


  0%|          | 0/103 [00:00<?, ?it/s]

Epoch: 12/60..  Training Loss: 0.340..  Test Loss: 0.247..  Test Accuracy: 0.931


  5%|▍         | 5/103 [00:20<07:01,  4.30s/it]

KeyboardInterrupt: ignored

In [0]:
model.cpu()
torch.save({'Epochs' : epochs,
           'state_dict' : model.state_dict()},
          'classifier.pth')


In [0]:
# Use it for Application

# Imports here
import matplotlib.pyplot as plt
import seaborn as sb

#%reload_ext autoreload        <------------— comment out 
#%autoreload 0                 <------------— comment out
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from torch.autograd import Variable
import json
from collections import OrderedDict

import math


def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    model = models.resnet152(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
        
    # Putting the classifier in pretrained network
    classifier = nn.Sequential(nn.Linear(2048 , 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 102),
                                 nn.LogSoftmax(dim=1))
    model.fc= classifier
    criterion = nn.NLLLoss()

    # Loading the weights
    model.load_state_dict(checkpoint['state_dict'],strict=False)
    model.eval()
    return model
# Load your model to this variable
model = load_checkpoint('/home/workspace/classifier.pth')   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

print('done')